In [1]:
# Imports
from collections import OrderedDict
import json
import pprint

## Assignments

### Input Assignments

In [49]:
# Json input
input_str= r"""
{
    "I1": "boolean",
    "I2": [ 
            "{\"1\", \"2\", \"3\"}",
            "{\"1\", \"2\"}",
            "{\"1\", \"2\", \"3\"}"
        ],
    "I3": [
            "0..10",
            5,
            "{0, 5, 7}"
        ],
    "STATUS": [
            "{\"stopped\", \"running\"}",
            "stopped",
            "{\"stopped\", \"running\"}"
        ]

}
"""

In [50]:
# Expected NuSMV
expected= 'init(I1):= {TRUE, FALSE};\n' + \
'init(I2):= {"1", "2"};\n' + \
'init(I3):= 5;\n' + \
'init(STATUS):= "stopped";\n' + \
'next(I1):= case\n' + \
'   stab: {TRUE, FALSE};\n' +\
'   TRUE: I1;\n' + \
'esac;\n' + \
'next(I2):= case\n' + \
'   stab: {"1", "2", "3"};\n' +\
'   TRUE: I2;\n' + \
'esac;\n'+ \
'next(I3):= case\n' + \
'   stab: {0, 5, 7};\n' +\
'   TRUE: I3;\n' + \
'esac;\n'+ \
'next(STATUS):= case\n' + \
'   stab: {"stopped", "running"};\n' +\
'   TRUE: STATUS;\n' + \
'esac;\n'

In [51]:
# Convert Json string to python dictionary
in_dict= json.loads(input_str, object_pairs_hook=OrderedDict)

In [52]:
in_dict

OrderedDict([('I1', 'boolean'),
             ('I2', ['{"1", "2", "3"}', '{"1", "2"}', '{"1", "2", "3"}']),
             ('I3', ['0..10', 5, '{0, 5, 7}']),
             ('STATUS',
              ['{"stopped", "running"}',
               'stopped',
               '{"stopped", "running"}'])])

In [62]:
# define initialization & assignment strings
init_str= ""
assign_str= ""

In [63]:
in_dict["I2"][1]

'{"1", "2"}'

In [64]:
# loop through elements
for key,value in in_dict.items():
    # Check if type is a boolean
    if isinstance(in_dict[key], str):
        # Initialization
        init_str= init_str + "init({0})".format(key) + ":= {TRUE, FALSE};\n"

        # Assignment
        assign_str= assign_str + \
            'next({0}):= case\n'.format(key) + \
            '   stab: {TRUE, FALSE};\n' +\
            '   TRUE: {0};\n'.format(key) + \
            'esac;\n'
        
    # if type is not a boolean
    else:

        # Initialization
        init_val= in_dict[key][1]
        # Check if initial value is a string and is not n enum type
        if (isinstance(init_val, str) and not ("{" in init_val)):
            init_val= '"' + init_val + '"'

        init_str= init_str + "init({0})".format(key) + ":= {0};\n".format(init_val)

        # Assignment
        assign_str= assign_str + \
            'next({0}):= case\n'.format(key) + \
            '   stab: {0};\n'.format(in_dict[key][2]) +\
            '   TRUE: {0};\n'.format(key) + \
            'esac;\n'
        
# return
out_str= init_str + assign_str

In [65]:
out_str= init_str + assign_str

In [66]:
out_str

'init(I1):= {TRUE, FALSE};\ninit(I2):= {"1", "2"};\ninit(I3):= 5;\ninit(STATUS):= "stopped";\nnext(I1):= case\n   stab: {TRUE, FALSE};\n   TRUE: I1;\nesac;\nnext(I2):= case\n   stab: {"1", "2", "3"};\n   TRUE: I2;\nesac;\nnext(I3):= case\n   stab: {0, 5, 7};\n   TRUE: I3;\nesac;\nnext(STATUS):= case\n   stab: {"stopped", "running"};\n   TRUE: STATUS;\nesac;\n'

In [67]:
pprint.pprint(out_str)

('init(I1):= {TRUE, FALSE};\n'
 'init(I2):= {"1", "2"};\n'
 'init(I3):= 5;\n'
 'init(STATUS):= "stopped";\n'
 'next(I1):= case\n'
 '   stab: {TRUE, FALSE};\n'
 '   TRUE: I1;\n'
 'esac;\n'
 'next(I2):= case\n'
 '   stab: {"1", "2", "3"};\n'
 '   TRUE: I2;\n'
 'esac;\n'
 'next(I3):= case\n'
 '   stab: {0, 5, 7};\n'
 '   TRUE: I3;\n'
 'esac;\n'
 'next(STATUS):= case\n'
 '   stab: {"stopped", "running"};\n'
 '   TRUE: STATUS;\n'
 'esac;\n')


In [68]:
pprint.pprint(expected)

('init(I1):= {TRUE, FALSE};\n'
 'init(I2):= {"1", "2"};\n'
 'init(I3):= 5;\n'
 'init(STATUS):= "stopped";\n'
 'next(I1):= case\n'
 '   stab: {TRUE, FALSE};\n'
 '   TRUE: I1;\n'
 'esac;\n'
 'next(I2):= case\n'
 '   stab: {"1", "2", "3"};\n'
 '   TRUE: I2;\n'
 'esac;\n'
 'next(I3):= case\n'
 '   stab: {0, 5, 7};\n'
 '   TRUE: I3;\n'
 'esac;\n'
 'next(STATUS):= case\n'
 '   stab: {"stopped", "running"};\n'
 '   TRUE: STATUS;\n'
 'esac;\n')


In [69]:
assert(out_str == expected)

### Place Assignment

In [42]:
# Json input
places_str= """
{
    "P1": [
        ["O1", "O2"],
        ["O3"]
    ],
    "P2":[
        ["O2", "O3"],
        ["O1", "O2"]
    ],
    "P3": [
        ["O1"],
        ["O2", "O3"]
    ],
    "initial": [ "P1", "P3" ]
    
}
"""

transitions_str= """
{
    "T1":[
        ["P1", "P2"],
        [
            [ ["I1", "I2"], ["I4"] ],
            [ ["I3"], ["I4"] ]
        ],
        ["P3"]
    ],
    "T2":[
        ["P3"],
        [
            [ ["I4"], ["I1", "I2"] ],
            [ ["I4"], ["I3"] ]
        ],
        ["P1", "P2"]
    ]
}
""" 

In [76]:
# Expected NuSMV
expected= 'init(P1):= true;\n' + \
    'init(P2):= false;\n' + \
    'init(P3):= true;\n' + \
    'next(P1):= case\n' + \
    '   T1: false;\n' +\
    '   T2: true;\n' +\
    '   true: P1;\n' + \
    'esac;\n' + \
    'next(P2):= case\n' + \
    '   T1: false;\n' +\
    '   T2: true;\n' +\
    '   true: P2;\n' + \
    'esac;\n' + \
    'next(P3):= case\n' + \
    '   T1: true;\n' +\
    '   T2: false;\n' +\
    '   true: P3;\n' + \
    'esac;\n'

In [44]:
# Convert Json string to python dictionary
places_dict= json.loads(places_str, object_pairs_hook=OrderedDict)
transitions_dict= json.loads(transitions_str, object_pairs_hook=OrderedDict)


In [72]:
# define initialization & assignment strings
init_str= ""
assign_str= ""

In [73]:
# Loop through places and initialize
for place_key,value in places_dict.items():
    if not (place_key == "initial"):
        # Initialization
        initial= "false"
        # Check if marked by default
        if place_key in places_dict["initial"]:
            initial= "true"

        init_str= init_str + "init({0}):= ".format(place_key) + \
            initial + ";\n"

        # Assignment
        assign_str= assign_str + \
                'next({0}):= case\n'.format(place_key) 
        # Loop through transitions
        for trans_key,value in transitions_dict.items():
            # Check if current place is a post-place of the current transition
            if place_key in transitions_dict[trans_key][2]:
                assign_str= assign_str + \
                '   {0}: true;\n'.format(trans_key)

            # Check if current place is a pre-place of the current transition
            if place_key in transitions_dict[trans_key][0]:
                assign_str= assign_str + \
                '   {0}: false;\n'.format(trans_key)

        assign_str= assign_str + \
                '   true: {0};\n'.format(place_key) + \
                'esac;\n'


In [74]:
pprint.pprint(init_str)

'init(P1):= true;\ninit(P2):= false;\ninit(P3):= true;\n'


In [75]:
pprint.pprint(assign_str)


('next(P1):= case\n'
 '   T1: false;\n'
 '   T2: true;\n'
 '   true: P1;\n'
 'esac;\n'
 'next(P2):= case\n'
 '   T1: false;\n'
 '   T2: true;\n'
 '   true: P2;\n'
 'esac;\n'
 'next(P3):= case\n'
 '   T1: true;\n'
 '   T2: false;\n'
 '   true: P3;\n'
 'esac;\n')
